In [1]:
from shared.preface import *
import shared.functions as fct

## Ch. 0: Preliminary Investigations.

### 2D grid, center coord. pairs.

In [ ]:
def grid_2D(l, s):

    # 2D spatial grid, discretized.
    eps = s/10
    x, y = np.mgrid[-l:l+eps:s, -l:l+eps:s]

    # [x y] edge coordinate pairs of above grid.
    xy = np.mgrid[-l:l+eps:s, -l:l+eps:s].reshape(2,-1).T

    # Create center coord.-pairs.
    x_centers = (x[1:,:] + x[:-1,:])/2.
    y_centers = (y[:,1:] + y[:,:-1])/2.
    centers = np.array([x_centers[:,:-1], y_centers[:-1,:]])
    cent_coordPairs2D = centers.reshape(2,-1).T
    print('All coord. pairs 2D:\n', cent_coordPairs2D)
    # print('Coord pairs 2D shape:', cent_coordPairs2D.shape)

    return cent_coordPairs2D

limit_coarse, space_coarse = 1.5, 1.
cent_coordPairs2D = grid_2D(limit_coarse, space_coarse)

# Delete middle square.
cent_coordPairs2D = np.delete(cent_coordPairs2D, 4, axis=0)

# Create finegrained square.
limit_fine, space_fine = limit_coarse/2., space_coarse/2.
cent_coordPairs2D_fine = grid_2D(limit_fine, space_fine)

# Insert finegrained square.

### 3D grid, center coord. pairs.

In [ ]:
# 3D spatial grid, discretized.
x, y, z = np.mgrid[-1:1.1:1., -1:1.1:1., -1:1.1:1.]
# print(x[0,...], x.shape)

x_centers = (x[1:,...] + x[:-1,...])/2.
# print(x_centers, x_centers.shape)

y_centers = (y[:,1:,:] + y[:,:-1,:])/2.
# print(y_centers, y_centers.shape)

z_centers = (z[...,1:] + z[...,:-1])/2.
# print(z_centers, z_centers.shape)


# Create center coord.-pairs., truncate redundant points.
centers3D = np.array([
    x_centers[:,:-1,:-1], 
    y_centers[:-1,:,:-1], 
    z_centers[:-1,:-1,:]
])
# print(centers3D, centers3D.shape)

cent_coordPairs3D = centers3D.reshape(3,-1).T 
print(cent_coordPairs3D, cent_coordPairs3D.shape)

## Ch. 1: Milky Way-type halo and simple grid.

### 3D plot of the DM particles.

In [ ]:
# Read DM particles.
fct.read_DM_positions_randomHalo(which_halos='halos', mass_select=12)

# Build grid around Milky Way.
MW_grid = fct.grid_3D(GRID_L, GRID_S) / kpc

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

# Read in DM particle positions.
DM_pos = np.load('sim_data/DM_positions_halos_M12.npy')
print(f'{len(DM_pos)}')
x_DM, y_DM, z_DM = DM_pos[:,0], DM_pos[:,1], DM_pos[:,2]
cut = 10
x, y, z = x_DM[1::cut], y_DM[1::cut], z_DM[1::cut]

ax.scatter(x, y, z, alpha=0.1, c='dodgerblue')

# Draw sphere around GC with radius=Rvir_MW.
rGC = Rvir_MW/kpc
uGC, vGC = np.mgrid[0:2 * np.pi:200j, 0:np.pi:100j]
xGC = rGC * np.cos(uGC) * np.sin(vGC)
yGC = rGC * np.sin(uGC) * np.sin(vGC)
zGC = rGC * np.cos(vGC)

xg, yg, zg = MW_grid[:,0], MW_grid[:,1], MW_grid[:,2] 
ax.scatter(xg, yg, zg, s=0.2, marker='x', color='black', alpha=0.5)


ax.plot_surface(
    xGC, yGC, zGC, alpha=0.1, 
    cmap=plt.cm.coolwarm, vmin=-1, vmax=1,# antialiased=False,
    rstride=1, cstride=1
)

plt.show()

### Calculate gravity in each cell.

In [ ]:
DM_pos = np.load('sim_data/DM_positions_halos_M12.npy')

### Testing 1 cell with coords. at earth.
cell_id = 0
cell1 = np.array([8.5, 0, 0])*kpc

cell_vector = fct.cell_gravity(cell1, DM_pos, GRAV_RANGE, DM_SIM_MASS)
cell_vector /= (kpc/s**2)  
print(cell_vector)
print(np.sqrt(np.sum(cell_vector**2)))

#! this should be around same order of magnitude as in spher. symmetric setup.

## Ch. 2: Broadcasting for cell_gravity.

In [ ]:
fct.read_DM_positions_randomHalo(which_halos='halos', mass_select=12)
DM = np.load('sim_data/DM_positions_halos_M12.npy')[1::10]
grid = fct.grid_3D(GRID_L, GRID_S)

grid = np.expand_dims(grid, axis=1)

DM = np.expand_dims(DM, axis=0)
DM = np.repeat(DM, len(grid), axis=0)
print('DM positions array shape', DM.shape)

# diff = DM - grid
# print(diff.shape, (diff.nbytes)/1e6)

fct.cell_gravity_3D(grid, DM, GRAV_RANGE, DM_SIM_MASS)
dPsi_grid = np.load('CubeSpace/dPsi_grid_snapsnot_X.npy')
dPsi_grid /= (kpc/s**2) 
mags = np.sqrt(np.sum(dPsi_grid**2, axis=1))
print(mags[0])

## Ch. 3: Combined precalculations for all snapshots.

In [ ]:
from shared.preface import *
import shared.functions as fct

for i in range(12,37):
    snap_i = f'{i:04d}'
    print(snap_i)

    # 1. Read in DM positions of halo in snapshot.
    fct.read_DM_positions_randomHalo(
        which_halos='halos', mass_select=12, mass_range=1., snap_num=snap_i
        )
    DM_pos = np.load('sim_data/DM_positions_halos_M12.npy')[1::10]

    # 2. Build the spatial grid, depending on virial radius of halo, etc.
    cell_grid = fct.grid_3D(GRID_L, GRID_S)
    np.save(f'CubeSpace/cell_grid_snapsnot_{snap_i}', cell_grid)
    cell_grid = np.expand_dims(cell_grid, axis=1)

    # 2.5 Adjust arrays.
    DM_pos = np.expand_dims(DM_pos, axis=0)
    DM_pos = np.repeat(DM_pos, len(cell_grid), axis=0)

    # 3. Calculate derivatives of each cell.
    fct.cell_gravity_3D(
        cell_grid, DM_pos, GRAV_RANGE, DM_SIM_MASS, snap_num=snap_i
        )

snapshot_0012 is at redshift ~3.8.

**BUT** it contains no halo of mass_select=12 anymore.

**TODO:** start at snapshot_0036 (z~1e-16) and **trace same halo** backwards through the
snapshots.

Ugly fix for now: Extended mass_select window to search down to 1e11*Msun halos.
Then just pick random halo, will not be the same in all snapshots.

### Redshift z of each snapshot.

In [ ]:
zeds = np.zeros(25)
nums = []
for j, i in enumerate(range(12,37)):
    snap_i = f'{i:04d}'
    nums.append(snap_i)

    with h5py.File(f'{SIM_DATA}/snapshot_{snap_i}.hdf5') as snap:
        zeds[j] = snap['Cosmology'].attrs['Redshift'][0]

np.save(f'shared/ZEDS_SNAPSHOTS.npy', np.asarray(zeds))
np.save(f'shared/NUMS_SNAPSHOTS.npy', np.asarray(nums))

## Ch. 4: Better derivative grid structure. 

### Final function: compile everything here later...

In [2]:
def DMnum_around_cell(cell_length):
    """Amount of DM particles in sphere around cell."""

    # Center all DM positions w.r.t. cell center.
    DM_cc = DM_coords*kpc - cell_coords

    # Calculate distances of DM to cc.
    DM_dist = np.sqrt(np.sum(DM_cc**2, axis=2))

    # Ascending order indices.
    ind = DM_dist.argsort(axis=1)
    ind_3D = np.expand_dims(ind, axis=2)
    ind_3D = np.repeat(ind_3D, 3, axis=2)

    # Sort DM positions according to dist.
    DM_pos_sort = np.take_along_axis(DM_cc, ind_3D, axis=1)
    DM_dist_sort = np.take_along_axis(DM_dist, ind, axis=1)

    # Keep DM inside certain range.
    DM_pos_inRange = DM_pos_sort[DM_dist_sort <= grav_range]


    # recursive structure most likely...
    # DMnum_around_cell(new_cell_length)
    # ...


### steps until: DM_pos array (cells, DM_amount, 3) containing nans.

In [3]:
# Random values for DM positions for testing.
cell_num, DM_num = 3, 6
DM_cc = np.random.random(size=(cell_num, DM_num, 3)) * 5
print(DM_cc)

# Calculate distances of DM to cc.
DM_dist = np.sqrt(np.sum(DM_cc**2, axis=2))

# Ascending order indices.
ind = DM_dist.argsort(axis=1)
ind_3D = np.expand_dims(ind, axis=2)
ind_3D = np.repeat(ind_3D, 3, axis=2)

# Sort DM positions according to dist.
DM_pos_sort = np.take_along_axis(DM_cc, ind_3D, axis=1)
DM_dist_sort = np.take_along_axis(DM_dist, ind, axis=1)

# Keep DM inside certain range.
radius = 5

# Index array for DM inside radius, for each cell: 
# [[cell_num, DM_particle_in_cell],...] (see print)
trunc_ind = np.argwhere(DM_dist_sort <= radius)
# print('trunc_ind \n', trunc_ind)

# Find index, up to which DM particles should be kept for each cell.
row = trunc_ind.T[1]
fw_diff = row[:-1] - row[1:]
take = np.argwhere(fw_diff.flatten() >= 0).flatten()

# -> Index up to max DM particle inside radius, paired with each cell num: 
# [[cell_0, DM_max_rank_for_cell_1],...] (see print)
selection = np.vstack((trunc_ind[take,:], trunc_ind[-1,:]))
print(selection, selection.shape)

# Replace all entries beyond these indices (for each cell) with nan values.
for i in range(cell_num):
    DM_cc[i, selection[i,1]+1:, :] = np.nan

print(DM_cc)

#! bug: some case not covered, where one cell does not get a selection index...
#!      run cell until you see error.

[[[1.77720521 1.13472618 0.56710239]
  [1.50666478 3.39075939 2.68714781]
  [0.63837537 0.49047689 3.48166293]
  [4.14749375 4.88532532 1.56511319]
  [3.26705726 1.52998272 3.61126162]
  [0.50974354 4.89114016 3.82863938]]

 [[1.99671012 0.78454745 2.66083601]
  [1.84396833 2.492121   4.84771411]
  [2.9629971  4.52293144 2.73252823]
  [3.12408156 1.01266043 4.11277781]
  [0.16491715 1.41447045 0.04733061]
  [2.71879643 3.98423678 3.42692945]]

 [[2.54738954 1.52687185 3.11092281]
  [1.30711265 2.7506287  3.95153528]
  [1.36032138 0.95882241 1.69491179]
  [1.16199381 1.55995133 0.42188494]
  [4.4323892  1.87168504 1.96340226]
  [0.57540819 1.38124483 2.7453328 ]]]
[[0 2]
 [1 1]
 [2 4]] (3, 2)
[[[1.77720521 1.13472618 0.56710239]
  [1.50666478 3.39075939 2.68714781]
  [0.63837537 0.49047689 3.48166293]
  [       nan        nan        nan]
  [       nan        nan        nan]
  [       nan        nan        nan]]

 [[1.99671012 0.78454745 2.66083601]
  [1.84396833 2.492121   4.84771411]
 

### steps until: deleting common nan rows, and cells not containing enough DM, and resetting DM positions.

In [4]:
# Drop all "rows", shared by all cells, which have only nan values,
# determined by maximum present value in DM rank of all cells.

# 1. Find max index (for DM rank) present in selection array.
max_DM_rank = np.max(selection[:,1])
print(max_DM_rank)

# 2. "Delete" those common rows. 
DM_cc_compact = np.delete(DM_cc, np.s_[max_DM_rank+1:], axis=1)

#! np. delete creates a new array, so delete old and free memory (just in case). 
# with contextlib.redirect_stdout(None):
#     del DM_cc
#     gc.collect()

print(DM_cc_compact)

# Counting how many (non-)nan rows are in each cell.
DM_particle_count = np.count_nonzero(~np.isnan(DM_cc_compact[:,:,0]), axis=1)
nan_count = np.count_nonzero(np.isnan(DM_cc_compact[:,:,0]), axis=1)
print(DM_particle_count, nan_count)


# Drop all cells, which have DM particle amount lower than threshold.
#note: see later which is more efficient, drop cells from DM_compact or before.
bool_DM = DM_particle_count < 2
print(bool_DM)

DM_cc_cell_filtered = np.delete(DM_cc_compact, bool_DM, axis=0)
print(DM_cc_cell_filtered)

# Also filter the cell coordinates. 
proxy_cell_coords = np.arange(cell_num*3).reshape((cell_num, 1, 3))
parent_cell_coords = np.delete(proxy_cell_coords, bool_DM, axis=0)

# Reset the DM coords. 
DM_cc_reset = DM_cc_cell_filtered + parent_cell_coords

4
[[[1.77720521 1.13472618 0.56710239]
  [1.50666478 3.39075939 2.68714781]
  [0.63837537 0.49047689 3.48166293]
  [       nan        nan        nan]
  [       nan        nan        nan]]

 [[1.99671012 0.78454745 2.66083601]
  [1.84396833 2.492121   4.84771411]
  [       nan        nan        nan]
  [       nan        nan        nan]
  [       nan        nan        nan]]

 [[2.54738954 1.52687185 3.11092281]
  [1.30711265 2.7506287  3.95153528]
  [1.36032138 0.95882241 1.69491179]
  [1.16199381 1.55995133 0.42188494]
  [4.4323892  1.87168504 1.96340226]]]
[3 2 5] [2 3 0]
[False False False]
[[[1.77720521 1.13472618 0.56710239]
  [1.50666478 3.39075939 2.68714781]
  [0.63837537 0.49047689 3.48166293]
  [       nan        nan        nan]
  [       nan        nan        nan]]

 [[1.99671012 0.78454745 2.66083601]
  [1.84396833 2.492121   4.84771411]
  [       nan        nan        nan]
  [       nan        nan        nan]
  [       nan        nan        nan]]

 [[2.54738954 1.52687185 3.

The array now contains all cells in need of subdivision, with coords. centered on (0,0,0). 

### steps for: creating coord array for sub 8 division of cells.

In [5]:
# Repeat each DM column 8 times, so each can get centered on a new cell.
DM_raw8 = np.repeat(DM_cc_reset, repeats=8, axis=0)

# Create 8 new cells around (0,0,0). Length and size of new cells determined
# by previous length of parent cell.
parent_GRID_S = 5
sub8 = fct.grid_3D(parent_GRID_S/2., parent_GRID_S/2)

# Match dimensions of current cell coords. array.
sub8 = np.expand_dims(sub8, axis=0)
sub8 = np.repeat(sub8, len(parent_cell_coords), axis=0)

# Center new "8-batch" of sub-cells on parent cell coords.
sub8_coords = sub8 - parent_cell_coords

# Now reshape array to match repeated DM position array.
sub8_coords = np.reshape(sub8_coords, (len(parent_cell_coords)*8, 3))
sub8_coords = np.expand_dims(sub8_coords, axis=1)

# Center the repeated DM columns on the different new sub8 cells.
DM_sub8_cc = DM_raw8 - sub8_coords

### Replace parent cells, which underwent division, with the 8 new coordinates.

Steps to think about & implement to complete algorithm:

1. the subdivision has to be done for many cells, each having different coords -> find broadcast-style method?
2. The coords. of the new sub-cells have to replace coords. for each parent cell. This has to be done before further subdivision happens, as cells with DM below threshhold get deleted from array.